#### Loading the Data

source: /raw_csv
target: /stagging

In [1]:
import duckdb
from pathlib import Path

In [2]:
data_folder = Path.cwd().parent / "data"
db_path = Path.cwd().parent / "hdbc.duckdb"

In [3]:
conn = duckdb.connect(str(db_path))

Load Tables

In [5]:
raw_guest = conn.sql(f"SELECT * FROM read_csv(?)", params= [str(data_folder / "raw_data" / "guest_info.csv")])


In [6]:
raw_booking = conn.sql(f"SELECT * FROM read_csv('{data_folder / "raw_data" /  'booking_details.csv'}')")

In [7]:
# raw_cancellation = conn.sql("SELECT * FROM read_csv('Data_Storage\\raw_csv\\cancellation_records.csv')")
raw_cancellation = conn.sql(f"SELECT * FROM read_csv('{data_folder / "raw_data" / 'cancellation_records.csv'}')")

In [7]:
conn.sql("CREATE TABLE stag_guest as (SELECT * FROM raw_guest)")

In [8]:
conn.sql("CREATE TABLE stag_booking as (SELECT * FROM raw_booking)")

In [9]:
conn.sql("CREATE TABLE stag_cancellation as (SELECT * FROM raw_cancellation)")

In [13]:
conn.sql(f"DESCRIBE stag_guest")

┌───────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│    column_name    │ column_type │  null   │   key   │ default │  extra  │
│      varchar      │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ booking_id        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ name              │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ email             │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ phone-number      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ credit_card       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ adults            │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ children          │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ babies            │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ customer_type     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ is_repeate

In [14]:
conn.sql(f"DESCRIBE stag_booking")

┌─────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│         column_name         │ column_type │  null   │   key   │ default │  extra  │
│           varchar           │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ booking_id                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ hotel                       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ lead_time                   │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ arrival_date_year           │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ arrival_date_month          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ arrival_date_week_number    │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ arrival_date_day_of_month   │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ stays_in_weekend_nights     │ BIGINT      │ YES     

In [15]:
conn.sql('show all tables')

┌──────────┬─────────┬───────────────────┬──────────────────────┬──────────────────────────────────────────┬───────────┐
│ database │ schema  │       name        │     column_names     │               column_types               │ temporary │
│ varchar  │ varchar │      varchar      │      varchar[]       │                varchar[]                 │  boolean  │
├──────────┼─────────┼───────────────────┼──────────────────────┼──────────────────────────────────────────┼───────────┤
│ hdbc     │ main    │ stag_booking      │ [booking_id, hotel…  │ [VARCHAR, VARCHAR, BIGINT, BIGINT, VAR…  │ false     │
│ hdbc     │ main    │ stag_cancellation │ [booking_id, is_ca…  │ [VARCHAR, BIGINT, VARCHAR, DATE, BIGIN…  │ false     │
│ hdbc     │ main    │ stag_guest        │ [booking_id, name,…  │ [VARCHAR, VARCHAR, VARCHAR, VARCHAR, V…  │ false     │
└──────────┴─────────┴───────────────────┴──────────────────────┴──────────────────────────────────────────┴───────────┘

Write a table to a CSV file

In [13]:
conn.execute(f"COPY stag_guest TO '{data_folder}/stag_data/stag_guest.csv' (FORMAT CSV, DELIMITER '|', HEADER)")

In [12]:
conn.execute(f"COPY stag_booking TO '{data_folder}/stag_data/stag_booking.csv'")


CREATE VIEW statement defines a new view in the catalog

In [4]:
conn.execute("CREATE VIEW v_stag_booking AS SELECT * FROM stag_booking")

CatalogException: Catalog Error: View with name "v_stag_booking" already exists!

Temporary Tables

In [6]:
t_booking = conn.execute(f"CREATE TEMP TABLE t_booking AS SELECT * FROM read_csv('{data_folder / 'raw_data' / 'booking_details.csv'}')")

In [8]:
conn.close()